# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
%pip install azureml-widgets
%pip install xgboost==1.5.0
%pip install --upgrade azureml-sdk[notebooks,automl]
%pip install azureml-train-automl-runtime==1.57.0
%pip install azureml-pipeline-core==1.57.0
%pip install azureml-pipeline-steps

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached azureml_automl_runtime-1.58.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached azureml_train_automl_runtime-1.58.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached azureml_training_tabular-1.58.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached azureml_interpret-1.58.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached azureml_mlflow-1.58.0-py3-none-any.whl.metadata (2.5 kB)
Using cached azureml_automl_runtime-1.58.0-py3-none-any.whl (1.8 MB)
Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
Using cached azureml_train_automl_runtime-1.58.0-py3-none-any.whl (420 kB)
Using cached azureml_interpret-1.58.0-py3-none-any.whl (52 kB)
Using cached azureml_mlflow-1.58.0-py3-none-any.whl (1.0 MB)
Using 

In [2]:
# %pip install cloudpickle==2.0.0
# %pip install azureml-dataprep
# %pip install azureml-automl-runtime==1.58.0

In [21]:
import logging
import os
import csv

import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.core import Workspace, Experiment, Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.58.0


In [27]:
import json
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
ws = Workspace.from_config(path="Users/odl_user_269062/config.json")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment

experiment_name = 'ml-experiment-project-final'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

quick-starts-ws-269118
aml-quickstarts-269118
westeurope
61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30


Name,Workspace,Report Page,Docs Page
ml-experiment-project-final,quick-starts-ws-269118,Link to Azure Machine Learning studio,Link to Documentation


In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "my-compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
import logging
import os
import csv

import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.58.0


In [7]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "survival-possibility-titanic"
description_text = "Possibility of a passenger can survive"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/hoangduchuy93/nd00333-capstone/refs/heads/main/train_survival.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,True,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,False,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    bool   
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: bool(1), float64(2), int64(4), object(5)
memory usage: 77.6+ KB


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Survived",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
ml-experiment-project-final,AutoML_d4036a96-a538-41cc-a788-3a89dac1e8cc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
ml-experiment-project-final,AutoML_d4036a96-a538-41cc-a788-3a89dac1e8cc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn mo

{'runId': 'AutoML_d4036a96-a538-41cc-a788-3a89dac1e8cc',
 'target': 'my-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-10-26T13:54:52.06826Z',
 'endTimeUtc': '2024-10-26T14:12:00.50802Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'my-compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-experiment-project-final","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group":"aml-quickstarts-269118","workspace_name":"quick-starts-ws-269118","region":"westeurope","compute_ta

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_auto_run, best_auto_model = remote_run.get_output()

In [13]:
best_auto_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/246f7a97/classes.interpret.json',
 'explanation/246f7a97/eval_data_viz.interpret.json',
 'explanation/246f7a97/expected_values.interpret.json',
 'explanation/246f7a97/features.interpret.json',
 'explanation/246f7a97/global_names/0.interpret.json',
 'explanation/246f7a97/global_rank/0.interpret.json',
 'explanation/246f7a97/global_values/0.interpret.json',
 'explanation/246f7a97/local_importance_values.interpret.json',
 'explanation/246f7a97/per_class_names/0.interpret.json',
 'explanation/246f7a97/per_class_rank/0.interpret.json',
 'explanation/246f7a97/per_class_values/0.interpret.json',
 'explanation/246f7a97/rich_metadata.interpret.json',
 'explanation/246f7a97/true_ys_viz.interpret.json',
 'explanation/246f7a97/visualization_dict.interpret.json',
 'explanation/246f7a97/ys_pred_proba_viz.interpret.json',
 'explanation/246f7a97/ys_pred_viz.interpret.json',
 'explanation/c628ff60/classes.interpret.json',
 'expl

In [15]:
# Save the environement from auto run
best_auto_run.download_file('outputs/conda_env_v_1_0_0.yml', "env.yml")

In [18]:
#TODO: Save the best model
register_best_model = best_auto_run.register_model(model_name="best_auto_model", 
model_path="outputs/model.pkl")

In [19]:
print(best_auto_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('5', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.5944444444444444, learning_rate=0.06316157894736842, max_bin=260, max_depth=6, min_child_weight=2, min_data_in_leaf=0.003457931034482759, min_split_gain=0.42105263157894735, n_estimators=10, n_jobs=1, num_leaves=161, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None, reg_alpha=0.3684210526315789, reg_lambda=0.15789473684210525, subsample=0.5447368421052632))], verbose=False)), ('25', Pipeline(memory=None, steps=[('maxabsscaler

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [20]:
# Download the score script from auto run
best_auto_run.download_file('outputs/scoring_file_v_1_0_0.py', "score.py")

In [24]:
inference_config = InferenceConfig( entry_script='score.py', 
                                    environment=best_auto_run.get_environment())
deploy_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                    memory_gb=1)
service=Model.deploy(ws, 
                        "deploy-automl-best-model", 
                        [register_best_model], 
                        inference_config, 
                        deploy_config)

service.wait_for_deployment(True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-10-26 14:47:22+00:00 Creating Container Registry if not exists..
2024-10-26 14:57:22+00:00 Registering the environment.
2024-10-26 14:57:24+00:00 Use the existing image..
2024-10-26 14:57:26+00:00 Submitting deployment to compute..
2024-10-26 14:57:32+00:00 Checking the status of deployment deploy-automl-best-model..
2024-10-26 15:02:17+00:00 Checking the status of inference endpoint deploy-automl-best-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [25]:
print(service.state)

Healthy


In [26]:
# Enable application insights
service.update(enable_app_insights=True)

In [30]:
# test
test = df.head(2)
ground_truth = test[["Survived"]]
test = test.drop("Survived", axis=1)
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,None,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C


In [40]:
print({"data": test.to_dict(orient="records")})

{'data': [{'PassengerId': 1, 'Pclass': 3, 'Name': 'Braund, Mr. Owen Harris', 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': None, 'Embarked': 'S'}, {'PassengerId': 2, 'Pclass': 1, 'Name': 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'Sex': 'female', 'Age': 38.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'PC 17599', 'Fare': 71.2833, 'Cabin': 'C85', 'Embarked': 'C'}]}


In [31]:
ground_truth

,Survived
0,False
1,True


In [42]:
testing_sample = json.dumps({"data": test.to_dict(orient="records")})
res = service.run(input_data=testing_sample)
print("Prediction: ", res)
print("Ground truth: ", ground_truth.to_dict(orient="records"))

Prediction:  {"result": [false, true]}
Ground truth:  [{'Survived': False}, {'Survived': True}]


TODO: In the cell below, print the logs of the web service and delete the service

In [35]:
print(service.get_logs())

2024-10-26T15:08:45,859165182+00:00 - rsyslog/run 
2024-10-26T15:08:45,871859358+00:00 - gunicorn/run 
2024-10-26T15:08:45,875056152+00:00 | gunicorn/run | 
2024-10-26T15:08:45,880845141+00:00 | gunicorn/run | ###############################################
2024-10-26T15:08:45,887171829+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-10-26T15:08:45,892059620+00:00 | gunicorn/run | ###############################################
2024-10-26T15:08:45,896709211+00:00 | gunicorn/run | 
2024-10-26T15:08:45,907123592+00:00 - nginx/run 
2024-10-26T15:08:45,912293182+00:00 | gunicorn/run | 
2024-10-26T15:08:45,930158249+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240918.v1
2024-10-26T15:08:45,934890140+00:00 | gunicorn/run | 
2024-10-26T15:08:45,939783831+00:00 | gunicorn/run | 
2024-10-26T15:08:45,951877308+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [43]:
service.delete()

Running
2024-10-26 16:23:59+00:00 Deleting service.
2024-10-26 16:24:03+00:00 Deleting service entity.
Succeeded


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
